In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline

In [2]:
train_raw = pd.read_csv('data_sets/train.csv')

In [3]:
foo = train_raw.copy()
foo['Age'] = foo['Age'].fillna(foo['Age'].mean())

In [4]:
def preprocess_categorical(data,column):
    processed = data.copy()
    for status in data[column].unique():
        processed[column+'_'+str(status)] = (data[column]==status).astype(int)
    processed = processed.drop(column,axis=1)
    return processed
def deck_letter(x):
    if type(x) == str:
        return x[0]
    else:
        return np.nan
def preprocess_cabin(data):
    data['Cabin'] = data['Cabin'].apply(deck_letter)
    return data
def preprocess_name(data):
    data['Title'] = data['Name'].apply(lambda x: x.split(',')[1].split()[0])
    return data    
def preprocess(data):
    data = preprocess_cabin(data)
    data = preprocess_name(data)
    categorical = ['Sex','Embarked','Cabin','Title']
    iters = 0
    while iters < len(categorical):
        data = preprocess_categorical(data,categorical[iters])
        iters += 1
    data = data.fillna(data.mean())
    return data
def preprocess_test(test,train):
    test = preprocess(test)
    for c_train in train.columns:
        if c_train not in test.columns:
            test[c_train] = 0
    for c_test in test.columns:
        if c_test not in train.columns:
            test = test.drop(c_test,axis=1)
    return test

In [5]:
train_p = preprocess(train_raw)
ignore = ['Name','Ticket','PassengerId']
train = train_p.drop(ignore,axis=1)

In [6]:
train_p.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Sex_female,Embarked_S,...,Title_Mme.,Title_Ms.,Title_Major.,Title_Lady.,Title_Sir.,Title_Mlle.,Title_Col.,Title_Capt.,Title_the,Title_Jonkheer.
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,...,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,0.647587,0.352413,0.722783,...,0.001122,0.001122,0.002245,0.001122,0.001122,0.002245,0.002245,0.001122,0.001122,0.001122
std,257.353842,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429,0.477990,0.477990,0.447876,...,0.033501,0.033501,0.047351,0.033501,0.033501,0.047351,0.047351,0.033501,0.033501,0.033501
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
train_x = train.drop(['Survived'],axis=1)
train_y = train['Survived'].values
train_x.describe()

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Sex_female,Embarked_S,Embarked_C,Embarked_Q,...,Title_Mme.,Title_Ms.,Title_Major.,Title_Lady.,Title_Sir.,Title_Mlle.,Title_Col.,Title_Capt.,Title_the,Title_Jonkheer.
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,...,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208,0.647587,0.352413,0.722783,0.188552,0.086420,...,0.001122,0.001122,0.002245,0.001122,0.001122,0.002245,0.002245,0.001122,0.001122,0.001122
std,0.836071,13.002015,1.102743,0.806057,49.693429,0.477990,0.477990,0.447876,0.391372,0.281141,...,0.033501,0.033501,0.047351,0.033501,0.033501,0.047351,0.047351,0.033501,0.033501,0.033501
min,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,29.699118,0.000000,0.000000,14.454200,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,35.000000,1.000000,0.000000,31.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
print train_x.columns
train_x.describe()

Index([u'Pclass', u'Age', u'SibSp', u'Parch', u'Fare', u'Sex_male',
       u'Sex_female', u'Embarked_S', u'Embarked_C', u'Embarked_Q',
       u'Embarked_nan', u'Cabin_nan', u'Cabin_C', u'Cabin_E', u'Cabin_G',
       u'Cabin_D', u'Cabin_A', u'Cabin_B', u'Cabin_F', u'Cabin_T',
       u'Title_Mr.', u'Title_Mrs.', u'Title_Miss.', u'Title_Master.',
       u'Title_Don.', u'Title_Rev.', u'Title_Dr.', u'Title_Mme.', u'Title_Ms.',
       u'Title_Major.', u'Title_Lady.', u'Title_Sir.', u'Title_Mlle.',
       u'Title_Col.', u'Title_Capt.', u'Title_the', u'Title_Jonkheer.'],
      dtype='object')


,Pclass,Age,SibSp,Parch,Fare,Sex_male,Sex_female,Embarked_S,Embarked_C,Embarked_Q,...,Title_Mme.,Title_Ms.,Title_Major.,Title_Lady.,Title_Sir.,Title_Mlle.,Title_Col.,Title_Capt.,Title_the,Title_Jonkheer.
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,...,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208,0.647587,0.352413,0.722783,0.188552,0.086420,...,0.001122,0.001122,0.002245,0.001122,0.001122,0.002245,0.002245,0.001122,0.001122,0.001122
std,0.836071,13.002015,1.102743,0.806057,49.693429,0.477990,0.477990,0.447876,0.391372,0.281141,...,0.033501,0.033501,0.047351,0.033501,0.033501,0.047351,0.047351,0.033501,0.033501,0.033501
min,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,29.699118,0.000000,0.000000,14.454200,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,35.000000,1.000000,0.000000,31.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
clf = AdaBoostClassifier(n_estimators=100)